In [1]:
import pandas as pd

In [60]:
#config
augmented_train_path = "../../data/dataframes/df_train_6C_augmented.csv"
augmented_test_path = "../../data/dataframes/df_test_6C_augmented.csv"
original_train_path = "../../data/dataframes/df_train_6C.csv"
original_test_path = "../../data/dataframes/df_test_6C.csv"
num_classes = 6

In [78]:
augmented_train_df = pd.read_csv(augmented_train_path)
augmented_test_df = pd.read_csv(augmented_test_path)
original_train_df = pd.read_csv(original_train_path)
original_test_df = pd.read_csv(original_test_path)

df_full = pd.read_csv("../../data/dataframes/all_patchs_dataframe_augmented.csv")

In [80]:
augmented_train_df.shape, augmented_test_df.shape, original_train_df.shape, original_test_df.shape, df_full.shape

((66649, 7), (16725, 7), (16703, 7), (4604, 7), (83374, 4))

In [68]:
# select the rows where the patch name are are valid 
regex = r'\d+_[A-Z]_row_\d+_col_\d+'

# valid of original test -> query set
original_test_valid_df = original_test_df[original_test_df['patch'].str.contains(regex)]
original_test_not_valid_df = original_test_df[~original_test_df['patch'].str.contains(regex)]

In [71]:
print('valid entries for the support set:', original_test_valid_df.shape[0])
print('not valid entries for the support set:', original_test_not_valid_df.shape[0])  

valid entries for the support set: 1696
not valid entries for the support set: 2908


In [72]:
# not valid of augmented test -> support set
augmented_test_valid_df = augmented_test_df[augmented_test_df['patch'].str.contains(regex)]
augmented_test_not_valid_df = augmented_test_df[~augmented_test_df['patch'].str.contains(regex)]

In [73]:
print('valid augmented entries for the support set:', augmented_test_valid_df.shape[0])
print('not valid augmented entries for the support set:', augmented_test_not_valid_df.shape[0])  

valid augmented entries for the support set: 6717
not valid augmented entries for the support set: 10008


In [88]:
# query dataset = valid of original test set
query_dataset_df = original_test_valid_df
query_dataset_df.shape[0]

1696

In [89]:
# support dataset = augmented train set + augmented not valid test set
support_dataset_df = pd.concat([augmented_train_df, augmented_test_not_valid_df])
support_dataset_df.shape[0]

76657

In [85]:
all_classes = df_full['classe'].unique()
all_labels = [0, 1, 2, 3, 4, 5]
all_classes, all_labels

(array(['Acinaire', 'Fibrose', 'Foetal', 'Glandulaire complexe',
        'Hémorragique', 'Lépidique', 'Micropapillaire', 'Mucineux',
        'Normal', 'Nécrose', 'Papillaire', 'Réactionnel', 'Solide',
        'Tissu lymphoïde'], dtype=object),
 [0, 1, 2, 3, 4, 5])

In [86]:
# check if query set have some missing classes
query_classes = query_dataset_df['classe'].unique()
missing_classes = [classe for classe in all_classes if classe not in query_classes]
missing_classes

['Foetal', 'Mucineux', 'Réactionnel', 'Solide']

In [87]:
# check if query set have some missing labels
query_labels = query_dataset_df['label'].unique()
missing_labels = [label for label in all_labels if label not in query_labels]
missing_labels

[]

## REDO THE SPLIT

In [95]:
df_full

,Patient,lame,patch,classe
0,13,D,13_D_row_83_col_32(1)_HF.jpg,Acinaire
1,2,B,2_B_row_4_col_6_HF.jpg,Acinaire
2,13,D,13_D_row_87_col_44.jpg,Acinaire
3,4,A,A_4_A (546).jpg,Acinaire
4,7,A,A_7_A (237)_HF.jpg,Acinaire
...,...,...,...,...
83369,7,A,7_A_row_129_col_66_HF.jpg,Tissu lymphoïde
83370,192,A,192_A_row_47_col_212_rot270.jpg,Tissu lymphoïde
83371,32,C,32_C_row_157_col_295_rot270.jpg,Tissu lymphoïde
83372,4,TL,TL_4_A (1342)_rot135.jpg,Tissu lymphoïde


In [104]:
df_full_valid = df_full[df_full['patch'].str.contains(regex)]
df_full_valid

,Patient,lame,patch,classe
0,13,D,13_D_row_83_col_32(1)_HF.jpg,Acinaire
1,2,B,2_B_row_4_col_6_HF.jpg,Acinaire
2,13,D,13_D_row_87_col_44.jpg,Acinaire
8,2,B,2_B_row_27_col_41_VF.jpg,Acinaire
9,13,D,13_D_row_97_col_25_VF.jpg,Acinaire
...,...,...,...,...
83365,226,C,226_C_row_87_col_239_rot225.jpg,Tissu lymphoïde
83369,7,A,7_A_row_129_col_66_HF.jpg,Tissu lymphoïde
83370,192,A,192_A_row_47_col_212_rot270.jpg,Tissu lymphoïde
83371,32,C,32_C_row_157_col_295_rot270.jpg,Tissu lymphoïde


In [90]:
df_full['classe'].value_counts()

classe
Réactionnel             18057
Fibrose                 13866
Nécrose                  9228
Hémorragique             6165
Solide                   5466
Normal                   5238
Tissu lymphoïde          4239
Lépidique                4095
Glandulaire complexe     4005
Papillaire               3450
Acinaire                 3396
Mucineux                 2466
Foetal                   2425
Micropapillaire          1278
Name: count, dtype: int64

In [110]:
# print the valid patients
all_patients = sorted(df_full['Patient'].unique())
valid_patients = sorted(df_full_valid['Patient'].unique())
print('all patients:', all_patients)
print('patients with valid data:', valid_patients)

all patients: [2, 3, 4, 5, 6, 7, 9, 13, 14, 16, 17, 22, 23, 32, 40, 44, 58, 72, 114, 133, 169, 192, 226, 230, 231, 247, 398]
patients with valid data: [2, 7, 13, 17, 32, 40, 44, 58, 133, 192, 226]


In [99]:
df_full[df_full['classe'] == 'Foetal']

,Patient,lame,patch,classe
17262,192,A,192_A_row_49_col_252_HF.jpg,Foetal
17263,169,Fo,Fo_169_A (136)_VF.jpg,Foetal
17264,192,A,192_A_row_51_col_246_rot90.jpg,Foetal
17265,169,Fo,Fo_169_A (23).jpg,Foetal
17266,169,Fo,Fo_169_A (20)_HF.jpg,Foetal
...,...,...,...,...
19682,192,A,192_A_row_62_col_250_VF.jpg,Foetal
19683,192,A,192_A_row_50_col_249.jpg,Foetal
19684,192,A,192_A_row_50_col_249_HF.jpg,Foetal
19685,192,A,192_A_row_57_col_291.jpg,Foetal


In [100]:
df_full[df_full['classe'] == 'Foetal']['Patient'].unique()

array([192, 169,   5])

In [102]:
df_full[df_full['Patient'] == 5]

,Patient,lame,patch,classe
49,5,A,A_5_A (1051)_HF.jpg,Acinaire
74,5,A,A_5_A (692)_HF.jpg,Acinaire
118,5,A,A_5_A (1084)_HF.jpg,Acinaire
186,5,A,A_5_A (484)_VF.jpg,Acinaire
191,5,A,A_5_A (448)_HF.jpg,Acinaire
...,...,...,...,...
82024,5,TL,TL_5_A (1554)_rot315.jpg,Tissu lymphoïde
82497,5,TL,TL_5_A (1510)_rot315.jpg,Tissu lymphoïde
82941,5,TL,TL_5_A (1554)_rot270.jpg,Tissu lymphoïde
83166,5,TL,TL_5_A (1510).jpg,Tissu lymphoïde


## Repeat for the training set

In [37]:
# select the rows where the patch name are are valid 
regex = r'\d+_[A-Z]_row_\d+_col_\d+'
train_df_valid = train_df[train_df['patch'].str.contains(regex)]
train_df_not_valid = train_df[~train_df['patch'].str.contains(regex)]
print(f'valid patches: {len(train_df_valid)}, invalid patches: {len(train_df_not_valid)}')

valid patches: 10291, invalid patches: 56358


In [38]:
train_df['classe'].value_counts(), train_df_valid['classe'].value_counts()

(classe
 Réactionnel             12777
 Fibrose                 12390
 Nécrose                  8994
 Solide                   5466
 Normal                   4857
 Hémorragique             3960
 Glandulaire complexe     3705
 Tissu lymphoïde          3366
 Mucineux                 2466
 Foetal                   2410
 Acinaire                 2205
 Lépidique                1833
 Micropapillaire          1170
 Papillaire               1050
 Name: count, dtype: int64,
 classe
 Hémorragique            3204
 Tissu lymphoïde         2034
 Lépidique               1371
 Acinaire                 936
 Mucineux                 732
 Foetal                   685
 Micropapillaire          657
 Normal                   525
 Papillaire                87
 Glandulaire complexe      48
 Fibrose                   12
 Name: count, dtype: int64)

In [39]:
# remove all items in the first array that are also in the second array
missing_valid_classes_train = train_df['classe'].value_counts().index.difference(train_df_valid['classe'].value_counts().index)
missing_valid_classes_train

Index(['Nécrose', 'Réactionnel', 'Solide'], dtype='object', name='classe')

## Repeat for whole dataset

In [41]:
df_full = pd.read_csv("../../data/dataframes/all_patchs_dataframe_augmented.csv")
df_full_valid = df_full[df_full['patch'].str.contains(regex)]

In [42]:
print(f'valid patches: {len(df_full_valid)}, invalid patches: {len(df_full) - len(df_full_valid)}')

valid patches: 17008, invalid patches: 66366


In [44]:
missing_valid_classes = df_full['classe'].value_counts().index.difference(df_full_valid['classe'].value_counts().index)
missing_valid_classes

Index(['Réactionnel', 'Solide'], dtype='object', name='classe')

Combined 

In [48]:
df_train_test = pd.concat([train_df, test_df])
df_train_test_valid = df_train_test[df_train_test['patch'].str.contains(regex)]
df_train_test

,Patient,lame,patch,classe,label,dataset,TetraClass
0,7,R,R_7_A (3844).jpg,Réactionnel,0,train,P
1,231,R,R_231_A (11)_VF.jpg,Réactionnel,0,train,P
2,231,R,R_231_A (16)_VF.jpg,Réactionnel,0,train,P
3,231,R,R_231_A (263).jpg,Réactionnel,0,train,P
4,7,R,R_7_A (2525)_VF.jpg,Réactionnel,0,train,P
...,...,...,...,...,...,...,...
16720,133,A,133_A_row_374_col_243_HF.jpg,Lépidique,5,test,T
16721,133,B,133_B_row_57_col_97_VF.jpg,Lépidique,5,test,T
16722,133,A,133_A_row_377_col_240_HF.jpg,Lépidique,5,test,T
16723,133,A,133_A_row_381_col_209.jpg,Lépidique,5,test,T


In [49]:
df_full_valid

,Patient,lame,patch,classe
0,13,D,13_D_row_83_col_32(1)_HF.jpg,Acinaire
1,2,B,2_B_row_4_col_6_HF.jpg,Acinaire
2,13,D,13_D_row_87_col_44.jpg,Acinaire
8,2,B,2_B_row_27_col_41_VF.jpg,Acinaire
9,13,D,13_D_row_97_col_25_VF.jpg,Acinaire
...,...,...,...,...
83365,226,C,226_C_row_87_col_239_rot225.jpg,Tissu lymphoïde
83369,7,A,7_A_row_129_col_66_HF.jpg,Tissu lymphoïde
83370,192,A,192_A_row_47_col_212_rot270.jpg,Tissu lymphoïde
83371,32,C,32_C_row_157_col_295_rot270.jpg,Tissu lymphoïde


In [51]:
missing_full_labels = df_train_test['label'].value_counts().index.difference(df_train_test_valid['label'].value_counts().index)
missing_full_labels

Index([], dtype='int64', name='label')